Question 1 Formulating the LP Model and Solving:


Objective function was :: 82*X1+102*X2+150*X3+105*X4+95*X5+84*X6+88*X7+125*x8+70*X9+64*X10


Constraints::

20*X1+25*X2+10*X3+30*X4+12.5*X5+35*X6+47.5*X7+10*X8+20*X9+50*X10 = 11687.5

75*X1+65*X2+70*X3+60*X4+85*X5+52.5*X6+42.5*X7+40*X8+35*X9+40*X10 = 24437.5

5*X1+10*X2+20*X3+10*X4+2.5*X5+12.5*X6+10*X7+50*X8+45*X9+10*x10 = 6375




In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.6 MB 15.4 MB/s 
     |████████████████████████████████| 49 kB 7.1 MB/s 


In [2]:
import numpy as np
from pyomo.environ import *

In [3]:
#intializing the model
model = ConcreteModel()
N = 10 #No of variables
M = 3 #No of constraints

row_indicies = np.arange(M)
col_indicies = np.arange(N)
#creating variables
model.x = Var(col_indicies)

In [4]:
#Constructing a two demionsal coefficient array
'''
   constraint 1 & 2 & 3 : this constraints are derived from the percentage of alloys required for chromium ,Aluminum and silver
''' 
constr_coef_A = np.array([
                          [20,25,10,30,12.5,35,47.5,10,20,50],
                          [75,65,70,60,85,52.5,42.5,40,35,40],
                          [5,10,20,10,2.5,12.5,10,50,45,10]
                          ])
#vector of rhs of constraints
constr_rhs_b = np.array([11687.5,24437.5,6375])
#vector for lower bound and upper bound
#lower bounds are taken from inhouse stock and there is no upper limit on purchasing outside so we have taken upper bound as np.inf
lower_bound = [15,16,9,17.5,12,18.5,2.5,0.5,9.5,8]
upper_bound = [np.inf]*10

In [5]:
#setting upper and lower bounds for variables
for i in col_indicies:
  model.x[i].setlb(lower_bound[i])
  model.x[i].setub(upper_bound[i])

In [6]:
#coef of cost function 
'''
we are considering  cost function as amount of cost required for outside purchase as we know inhouse stock and its cost so we are subracting it from objective function 

'''
obj_coef_c = np.array([82,102,150,105,95,84,88,125,70,64])
inhouse_stock_price = 15*(82-45)+16*(102-48)+9*(150-55)+17.5*(105-65)+12*(95-54)+18.5*(84-37.5)+2.5*(88-35)+0.5*(125-52.5)+9.5*(70-47)+8*(64-28)
#creating object function ,sense = minimize(by default)
model.objective = Objective(expr = sum(model.x[i]*obj_coef_c[i] for i in col_indicies)-inhouse_stock_price )

In [7]:
model.constraints = ConstraintList()
#adding all constraints using for loop
for i in row_indicies:
  model.constraints.add(sum(model.x[j]*constr_coef_A[i][j] for j in col_indicies)==constr_rhs_b[i])

In [8]:
#printing the model
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :    15 :  None :  None : False :  True :  Reals
          1 :    16 :  None :  None : False :  True :  Reals
          2 :     9 :  None :  None : False :  True :  Reals
          3 :  17.5 :  None :  None : False :  True :  Reals
          4 :    12 :  None :  None : False :  True :  Reals
          5 :  18.5 :  None :  None : False :  True :  Reals
          6 :   2.5 :  None :  None : False :  True :  Reals
          7 :   0.5 :  None :  None : False :  True :  Reals
          8 :   9.5 :  None :  None : False :  True :  Rea

In [9]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155680 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [10]:
SolverFactory('glpk',executable = '/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 28747.1875
  Upper bound: 28747.1875
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 11
  Number of nonzeros: 31
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.014745950698852539
# ----------------------------------------------------------
#   Solution Information
# ------------------------

In [11]:
#printing values of constraints at optimal value
print("\n Constraints value at optimal values of variable")
model.constraints.display()


 Constraints value at optimal values of variable
constraints : Size=3
    Key : Lower   : Body    : Upper
      1 : 11687.5 : 11687.5 : 11687.5
      2 : 24437.5 : 24437.5 : 24437.5
      3 :  6375.0 :  6375.0 :  6375.0


In [12]:
#displaying solution 
print("\nOptimal cost function value = ",model.objective())

#display optimal values of variables

for i in col_indicies:
  print(f'X{i} = ',model.x[i].value)


Optimal cost function value =  28747.1875
X0 =  173.078125
X1 =  16.0
X2 =  9.0
X3 =  17.5
X4 =  12.0
X5 =  18.5
X6 =  2.5
X7 =  0.5
X8 =  83.546875
X9 =  92.375


Question 3 Solution

So Optimal cost value of the lpp is 28747.1875

Optimal values of variables

* X0 =  173.078125
* X1 =  16.0
* X2 =  9.0
* X3 =  17.5
* X4 =  12.0
* X5 =  18.5
* X6 =  2.5
* X7 =  0.5
* X8 =  83.546875
* X9 =  92.375

Question 4 Solution

Amount of alloys that are needed to be bought from market

* X0 =  173.078125-15 = 158.078125
* X1 =  16.0-16 = 0
* X2 =  9.0-9 = 0
* X3 =  17.5-17.5 = 0
* X4 =  12.0-12 = 0
* X5 =  18.5-18.5 = 0
* X6 =  2.5 -2.5 = 0
* X7 =  0.5 -0.5 = 0
* X8 =  83.546875 -9.5 = 74.046875
* X9 =  92.375 -8 = 84.375

Top 3 alloys was A1>A10>A9
Bottom 3 Alloys are A2 = A3 = A4 = A5 = A6 = A7 = A8 = 0


Question 5 SOlution

The Alloys for which inhouse stock was completly used  is A1,A2,A3,A4,A5,A6.A7,A8,A9,A10 

Yes inhouse stock of all alloys is completly used beacause we first used the inhouse stock and then moved to purchase outside stock


Question 6 Solution

The Alloys which are not required to buy from market is A2,A3,A4,A5,A6,A7,A8

Because we don't require more than the inhouse stock for the above variables 

as we first prefer to use inhouse stock rather than outside stock



In [13]:
#printing values of constraints at optimal value
print("\n Constraints value at optimal values of variable")
model.constraints.display()


 Constraints value at optimal values of variable
constraints : Size=3
    Key : Lower   : Body    : Upper
      1 : 11687.5 : 11687.5 : 11687.5
      2 : 24437.5 : 24437.5 : 24437.5
      3 :  6375.0 :  6375.0 :  6375.0




> Indented block


